In [1]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### GGF HH production

In [2]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     processParameters = cms.vstring(
                                                     __CHANNEL_DECAY_FRAGMENT__),
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 'processParameters'
                                                                                 )
                                                     )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [3]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2016APV': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [4]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(True),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [5]:
years = ["2016", "2016APV", "2017", "2018"]
processes = ["SM","1","2","3","4","5","6","7","8","9","10","11","12"]
#mass_point = ["250","260","270","280","300","320","350","400","450","500","550","600","650","700","750","800","850","900","1000","1250","1500","1750","2000","2500","3000"]
tot_events = 400000
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [6]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/GF_HH_benchmarks'
    t_datasetname_year = 'GluGluToHHTo2B2Tau_TuneCP5_PSWeights_node_{process}_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/doubleHiggs/GF_HH_benchmarks/GF_HH_{process}_slc7_amd64_gcc700_CMSSW_9_3_16_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    

    for process in range(len(processes)):
        #gp_ggf_year = "gp_ggf_" + years[year]
        dataset_names_year = []
        gp_ggf_year = []
        d = {"process":processes[process]}
        tmp_example_dict[processes[process]] = t_example1
        gp_ggf_year.append(t_gp_year.format_map(d))
        dataset_names_year.append(t_datasetname_year.format_map(d))
        tmp_dataset_dict[processes[process]] = dataset_names_year
        tmp_gridpack_dict[processes[process]] = gp_ggf_year
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

print(dataset_names['2016']['1'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

['GluGluToHHTo2B2Tau_TuneCP5_PSWeights_node_1_13TeV-madgraph-pythia8']


In [7]:
ggf_hh_bbtt = '''                                                                                                                                                                                           
'25:m0 = 125.0',
'25:onMode = off',
'25:onIfMatch = 5 -5',
'25:onIfMatch = 15 -15',
'ResonanceDecayFilter:filter = on',
'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
'ResonanceDecayFilter:mothers = 25', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
'ResonanceDecayFilter:daughters = 5,5,15,15',
'''
#'SpaceShower:dipoleRecoil = on' 


## .csv Production

In [8]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
for process in processes:
    tmp_process_pythia_map[process] = fragmentsDictCreator(ggf_hh_bbtt)
process_pythia_map.update(tmp_process_pythia_map)

In [9]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('ggF_'+year+'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events','fragment','notes','Generator','mcdbid','time','size'])
        for process in processes:
            tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
            version="2.6.5"
            dataset_name = dataset_names[year][process][0]
            if year == "2016APV":
                events = round(tot_events*0.54)
            elif year == "2016":
                events = round(tot_events*0.46)
            elif year == "2017":
                events = tot_events
            elif year == "2018":
                events = tot_events
            print(process_pythia_map[process][year])
            tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process][0]) + '\n' + process_pythia_map[process][year]
            final_fragment = tmp_fragment.replace('__EXAMPLE__', example_dict[year][process])
            note = dataset_name.replace('_',' ')
            generators="Madgraph_" + version + "  Pythia8"
            mcdb_id = '0'
            time = '0.3'
            size = '120'
            csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])


from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     processParameters = cms.